In [6]:
# --- Comprehensive Demonstration of OACDT Properties ---
import random
import OACDT as treap
from collections import Counter

if __name__ == '__main__':
    print("="*60)
    print(" COMPREHENSIVE DEMONSTRATION OF OACDT PROPERTIES")
    print("="*60)

    # 1. INITIALIZATION
    # We create a system with 32 physical cache nodes and a replication factor of 5.
    NUM_NODES = 32
    REPLICA_FACTOR = 5
    oacdt = treap.OACDT(num_nodes=NUM_NODES, replica_factor=REPLICA_FACTOR)
    oacdt.verbose = True # Enable detailed logging for rotations
    print(f"\n[1. Initialization] System initialized with {NUM_NODES} nodes and replica factor {REPLICA_FACTOR}.")

    # 2. DETERMINISTIC MAPPING PROPERTY (from Orthogonal Array concept)
    # The same key MUST always map to the same set of replica nodes.
    print("\n[2. Deterministic Mapping] Demonstrating that a key always maps to the same replicas.")
    key_for_test = "user:profile:alpha"
    replicas1 = oacdt._get_replicas_from_oa(key_for_test)
    replicas2 = oacdt._get_replicas_from_oa(key_for_test)
    print(f" -> First mapping for '{key_for_test}': {replicas1}")
    print(f" -> Second mapping for '{key_for_test}': {replicas2}")
    assert replicas1 == replicas2
    print(" -> SUCCESS: Both mappings are identical as required.")

    # 3. UNIFORM DISTRIBUTION PROPERTY (from Orthogonal Array concept)
    # A large number of keys should be distributed roughly evenly across all nodes.
    print("\n[3. Uniform Distribution] Analyzing replica distribution for 1000 keys.")
    distribution_counter = Counter()
    for i in range(1000):
        replicas = oacdt._get_replicas_from_oa(f"key-{i}")
        distribution_counter.update(replicas)
    
    total_assignments = sum(distribution_counter.values())
    expected_per_node = total_assignments / NUM_NODES
    print(f" -> Total replica assignments: {total_assignments}")
    print(f" -> Expected assignments per node (ideal): {expected_per_node:.2f}")
    print(" -> Actual distribution (Node ID: Count):")
    for node_id, count in sorted(distribution_counter.items()):
        print(f"    Node {node_id:2d}: {count} assignments")
    print(" -> OBSERVATION: The load is spread across all nodes with no major 'hot spots'.")

    # 4. TREAP OPERATIONS & PROPERTIES (BST + HEAP)
    print("\n[4. Treap Operations] Demonstrating Insert, Search, Update, Delete, and BST property.")
    
    # 4a. Insertion
    print("\n -> [4a. Insertion] Inserting initial key-value pairs...")
    data_to_insert = {
        10: "apple", 20: "banana", 5: "cherry", 15: "date", 25: "elderberry"
    }
    for key, val in data_to_insert.items():
        print(f"  - Inserting key: {key}")
        oacdt.insert(key, val)

    # 4b. BST Property (Sorted Order)
    print("\n -> [4b. BST Property] Inorder traversal shows keys are sorted.")
    traversal = oacdt.inorder_traversal()
    print(f"  - Traversal result: {traversal}")
    sorted_keys = [item[0] for item in traversal]
    assert sorted_keys == sorted(data_to_insert.keys())
    print(" -> SUCCESS: Keys are in sorted order, confirming BST property.")

    # 4c. Search & Fault Tolerance Demonstration
    print("\n -> [4c. Search & Fault Tolerance] Searching for an existing key.")
    key_to_find = 15
    replicas = oacdt._get_replicas_from_oa(key_to_find)
    value = oacdt.search(key_to_find)
    print(f"  - Searched for key '{key_to_find}', found value: '{value}'")
    print(f"  - This data is replicated on nodes: {replicas}")
    print(f"  - FAULT TOLERANCE: If node {replicas[0]} fails, the data can still be retrieved from {replicas[1:]}.")
    
    # 4d. Update an existing key
    print("\n -> [4d. Update] Updating the value for an existing key.")
    key_to_update = 10
    print(f"  - Value of key '{key_to_update}' before update: '{oacdt.search(key_to_update)}'")
    oacdt.insert(key_to_update, "apricot") # Inserting with same key updates value
    print(f"  - Value of key '{key_to_update}' after update: '{oacdt.search(key_to_update)}'")
    assert oacdt.search(key_to_update) == "apricot"
    print(" -> SUCCESS: Value was updated correctly.")

    # 4e. Deletion
    print("\n -> [4e. Deletion] Deleting a key from the treap.")
    key_to_delete = 20
    print(f"  - Deleting key '{key_to_delete}'...")
    oacdt.delete(key_to_delete)
    print(f"  - Inorder traversal after deletion: {oacdt.inorder_traversal()}")
    assert oacdt.search(key_to_delete) is None
    print(f" -> SUCCESS: Key '{key_to_delete}' was removed.")

    # 5. SCALABILITY DEMONSTRATION (Handling more data)
    print("\n[5. Scalability] Inserting 100 more random keys to show system stability.")
    for _ in range(100):
        random_key = random.randint(100, 1000)
        random_value = f"random_data_{random_key}"
        oacdt.insert(random_key, random_value)
    print(" -> Successfully inserted 100 random keys.")
    print(f" -> Total items in Treap now: {len(oacdt.inorder_traversal())}")
    print(" -> OBSERVATION: The OACDT handles a larger dataset due to the O(log N) complexity of Treap operations.")

    print("\n" + "="*60)
    print(" DEMONSTRATION COMPLETE")
    print("="*60)



 COMPREHENSIVE DEMONSTRATION OF OACDT PROPERTIES

[1. Initialization] System initialized with 32 nodes and replica factor 5.

[2. Deterministic Mapping] Demonstrating that a key always maps to the same replicas.
 -> First mapping for 'user:profile:alpha': [26, 27, 28, 29, 30]
 -> Second mapping for 'user:profile:alpha': [26, 27, 28, 29, 30]
 -> SUCCESS: Both mappings are identical as required.

[3. Uniform Distribution] Analyzing replica distribution for 1000 keys.
 -> Total replica assignments: 5000
 -> Expected assignments per node (ideal): 156.25
 -> Actual distribution (Node ID: Count):
    Node  0: 148 assignments
    Node  1: 164 assignments
    Node  2: 157 assignments
    Node  3: 165 assignments
    Node  4: 148 assignments
    Node  5: 142 assignments
    Node  6: 138 assignments
    Node  7: 138 assignments
    Node  8: 137 assignments
    Node  9: 151 assignments
    Node 10: 171 assignments
    Node 11: 175 assignments
    Node 12: 180 assignments
    Node 13: 185 assignme